In [1]:
from pydantic import BaseModel, Field, ValidationError

In [2]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=5)

In [3]:
from typing import Annotated

BoundInt = Annotated[int, Field(gt=0, lt=5)]

class Model(BaseModel):
    number: BoundInt

In [4]:
from datetime import datetime
from typing import Any

from dateutil.parser import parse

def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValidationError(str(ex))
    return value

In [5]:
from pydantic import BeforeValidator

In [6]:
DateTime = Annotated[datetime, BeforeValidator(parse_datetime)]

class Model(BaseModel):
    dt: DateTime

In [7]:
Model(dt="2020/1/1 1pm")

Model(dt=datetime.datetime(2020, 1, 1, 13, 0))

In [10]:
import pytz

def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt

In [11]:
from pydantic import AfterValidator

In [12]:
DateTime = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

In [13]:
class Model(BaseModel):
    dt: DateTime

In [14]:
Model(dt="2020/1/1 1pm")

Model(dt=datetime.datetime(2020, 1, 1, 13, 0, tzinfo=<UTC>))

In [15]:
eastern = pytz.timezone("US/Eastern")
dt = eastern.localize(datetime(2020, 1, 1, 3, 0, 0))
dt

datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [16]:
Model(dt=dt)

Model(dt=datetime.datetime(2020, 1, 1, 8, 0, tzinfo=<UTC>))

In [17]:
def before_validator_1(value):
    print("before_validator_1")
    return value

def before_validator_2(value):
    print("before_validator_2")
    return value

def before_validator_3(value):
    print("before_validator_3")
    return value

def after_validator_1(value):
    print("after_validator_1")
    return value

def after_validator_2(value):
    print("after_validator_2")
    return value

def after_validator_3(value):
    print("after_validator_3")
    return value

In [18]:
CustomType = Annotated[
    int,
    BeforeValidator(before_validator_1),
    BeforeValidator(before_validator_2),
    AfterValidator(after_validator_1),
    AfterValidator(after_validator_2),
    BeforeValidator(before_validator_3),
    AfterValidator(after_validator_3)
]

In [19]:
class Model(BaseModel):
    number: CustomType

In [20]:
Model(number=10)

before_validator_3
before_validator_2
before_validator_1
after_validator_1
after_validator_2
after_validator_3


Model(number=10)

In [21]:
def are_elements_unique(values: list[Any]) -> list[Any]:
    unique_elements = []
    for value in values:
        if value in unique_elements:
            raise ValueError("elements must be unique")
        unique_elements.append(value)
    return values

In [22]:
UniqueIntList = Annotated[
    list[int], AfterValidator(are_elements_unique)
]

In [23]:
class Model(BaseModel):
    numbers: UniqueIntList = []

In [24]:
Model(numbers=[1, 2, 3, 4])

Model(numbers=[1, 2, 3, 4])

In [26]:
try:
    Model(numbers=[1, 2, 3, 2])
except ValidationError as ex:
    print(ex)

1 validation error for Model
numbers
  Value error, elements must be unique [type=value_error, input_value=[1, 2, 3, 2], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [27]:
from typing import TypeVar
T = TypeVar("T")

In [28]:
UniqueList = Annotated[list[T], AfterValidator(are_elements_unique)]

In [33]:
class Model(BaseModel):
    elements: UniqueList

In [34]:
Model(elements=[1, 2, 3])

Model(elements=[1, 2, 3])

In [37]:
Model(elements={'a', 'b', 'c'})

Model(elements=['b', 'a', 'c'])

In [39]:
try:
    Model(elements=['a', 'b', 'b'])
except ValidationError as ex:
    print(ex)

1 validation error for Model
elements
  Value error, elements must be unique [type=value_error, input_value=['a', 'b', 'b'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [40]:
UniqueList = Annotated[
    list[T],
    Field(min_length=2, max_length=5),
    AfterValidator(are_elements_unique)
    ]

In [41]:
class Model(BaseModel):
    numbers: UniqueList[int] = []
    strings: UniqueList[str] = []

In [42]:
Model(numbers=[1, 2, 3], strings=['a', 'b', 'c'])

Model(numbers=[1, 2, 3], strings=['a', 'b', 'c'])

In [43]:
try:
    Model(numbers=[1, ] * 5, strings=str("python"))
except ValidationError as ex:
    print(ex)

2 validation errors for Model
numbers
  Value error, elements must be unique [type=value_error, input_value=[1, 1, 1, 1, 1], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
strings
  Input should be a valid list [type=list_type, input_value='python', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/list_type
